In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Feb  4 20:15:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    27W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip -q install tensorflow tensorflow-gpu opencv-python matplotlib keras

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tensorflow-gpu
  error: subprocess-exited-with-error
  
  × Running setup.py install for tensorflow-gpu did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Running setup.py install for tensorflow-gpu ... error
error: legacy-install-failure

× Encountered error while trying to install package.
╰─> tensorflow-gpu

note: This is an issue with the package mentioned above, not pip.
hint: See above for output from the failure.


In [3]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [4]:
import os
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from keras import layers
import matplotlib
import pathlib
import cv2
import imghdr
import time
from matplotlib import pyplot as plt
import numpy as np
from keras.models import Sequential, load_model # MAY NEED TO BE REPLACED W/ FUNCTIONAL API
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, AveragePooling2D
from keras.metrics import Precision, Recall, BinaryAccuracy


In [5]:
# !ls 'drive/MyDrive/Colab Notebooks/Polyamanita/data/train/'

In [6]:
model_name = 'train_orig_9'
model_folder = 'drive/MyDrive/Colab Notebooks/Polyamanita/models/' + model_name
data_folder = 'drive/MyDrive/Colab Notebooks/Polyamanita/data/train_orig_9'

In [7]:
model = tf.keras.models.load_model(model_folder + '/savedmodel.h5', compile = True) # Load the model from the .h5 file
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 512, 512, 3)       0         
                                                                 
 rescaling_2 (Rescaling)     (None, 512, 512, 3)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 512, 512, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 256, 256, 16)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 256, 256, 32)      4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 128, 128, 32)     0         
 2D)                                                  

In [8]:
dataset = tf.keras.utils.image_dataset_from_directory(
    data_folder,
    shuffle = False,
    image_size = (512, 512),
    batch_size = 32,
    labels='inferred',
    label_mode = 'categorical'
)

Found 4500 files belonging to 9 classes.


In [9]:
dir_order = os.listdir(data_folder)
Y = []
for dir in dir_order:
    num_files = len(os.listdir(data_folder + '/' + dir))
    for i in range(num_files):
        Y.append(dir)

print(Y[0])

Clathrus_columnatus


In [10]:
predictions = model.predict(dataset) # THIS TAKES A WHILE
np.save(model_folder + '/pred_temp', predictions)

141/141 [==============================] - 1081s 8s/step


In [11]:
predictions = np.load(model_folder + '/pred_temp.npy')

In [12]:
classes = dataset.class_names
Y_pred = np.array([classes[np.argmax(vec)] for vec in predictions])
print(Y_pred[:5])
print(len(Y_pred))

['Chlorophyllum_molybdites' 'Lactarius_indigo' 'Ganoderma_curtisii'
 'Ganoderma_curtisii' 'Ganoderma_curtisii']
4500


In [13]:
results = pd.DataFrame({'Y': Y, 'Y_pred': Y_pred})
results['match'] = (results['Y'] == results['Y_pred']) # Generate matching column
print(results)

                        Y                    Y_pred  match
0     Clathrus_columnatus  Chlorophyllum_molybdites  False
1     Clathrus_columnatus          Lactarius_indigo  False
2     Clathrus_columnatus        Ganoderma_curtisii  False
3     Clathrus_columnatus        Ganoderma_curtisii  False
4     Clathrus_columnatus        Ganoderma_curtisii  False
...                   ...                       ...    ...
4495   Ganoderma_curtisii         Lentinus_tigrinus  False
4496   Ganoderma_curtisii          Lactarius_indigo  False
4497   Ganoderma_curtisii          Lactarius_indigo  False
4498   Ganoderma_curtisii        Pisolithus_arhizus  False
4499   Ganoderma_curtisii        Pisolithus_arhizus  False

[4500 rows x 3 columns]


In [14]:
total_correct = results['match'].sum()
print('Correct:\t', total_correct, '\nIncorrect:\t', results.shape[0] - total_correct)
accuracy = (results['match'].sum() / results.shape[0]) * 100
print(f'Accuracy:\t {round(accuracy, 3)}%')

Correct:	 795 
Incorrect:	 3705
Accuracy:	 17.667%


In [15]:
class_results = pd.DataFrame({'class': classes})
class_results['total_correct'] = 0
class_results['total_incorrect'] = 0
class_results['total'] = 0
# print(class_results)

In [ ]:
for i in range(results.shape[0]):
    class_name = results['Y'].loc[i]
    row_idx = class_results.loc[class_results['class'] == class_name].index
    if results.loc[i, 'match']:
        class_results.loc[row_idx, 'total_correct'] += 1
    else:
        class_results.loc[row_idx, 'total_incorrect'] += 1

    class_results.loc[row_idx, 'total'] += 1
print(class_results)

In [ ]:
class_results['accuracy'] = class_results['total_correct'] / class_results['total']
print(class_results)

In [ ]:
class_results.to_csv(model_folder + '/class_analysis.csv')